In [1]:
!pip install transformers --quiet
!pip install sentencepiece --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls 'drive/MyDrive/Kuliah/Semester 7/Pemrosesan Bahasa Alami/Tugas besar'

 dataset			  ppt.pptx
 datauji-Question-generator.csv  'Question generator Bert.ipynb'
 hasiluji-Question-generator-T5  'Question generator main '
 model				 'Question generator T5.ipynb'


In [4]:
import torch
import transformers
from transformers import BertLMHeadModel, BertConfig, BertTokenizer
import pandas as pd

In [5]:
DIR = 'drive/MyDrive/Kuliah/Semester 7/Pemrosesan Bahasa Alami/Tugas besar/'
SEQ_LENGTH = 512
PRETRAINED_MODEL = 'bert-base-uncased'

In [6]:
df = pd.read_csv(DIR + 'dataset/SQuAD-v1.1.csv')

In [7]:
df = df.dropna()
df.isnull().values.any()

False

In [8]:
dataset = df[533:633].drop(columns=['title', 'answer_start', 'answer_end', 'question']).reset_index()
dataset.head()

,index,context,answer
0,533,"On January 7, 2012, Beyoncé gave birth to her ...","January 7, 2012"
1,534,"On January 7, 2012, Beyoncé gave birth to her ...",Blue Ivy Carter
2,535,"On January 7, 2012, Beyoncé gave birth to her ...",Lenox Hill Hospital in New York.
3,536,"On January 7, 2012, Beyoncé gave birth to her ...",Revel Atlantic City's Ovation Hall
4,537,"On January 7, 2012, Beyoncé gave birth to her ...",four


In [9]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

2

In [10]:
saved = torch.load(DIR + 'model/qg_pretrained_bert_model_trained.pth', map_location='cpu')
config = BertConfig(decoder_start_token_id=tokenizer.pad_token_id)
config.is_decoder = True
model = BertLMHeadModel(config).from_pretrained(PRETRAINED_MODEL)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(saved['model_state_dict'])

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [11]:
def inference(dataset):
  input_texts = '<answer> ' + dataset['answer'] + ' <context> ' + dataset['context']
  input_sequences = tokenizer(
            input_texts.tolist(), 
            padding='max_length', 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
  
  output_sequences = model.generate(
      input_ids=input_sequences['input_ids'],
      attention_mask=input_sequences['attention_mask']
  )

  questions = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
  
  return pd.DataFrame({
      'context': dataset['context'],
      'question': questions,
      'answer': dataset['answer']
  })



In [12]:
results = inference(dataset)

Input length of input_ids is 512, but ``max_length`` is set to 20. This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


In [13]:
results.to_csv(DIR + 'hasiluji-Question-generator-bert.csv')